# This notebook will use sequential model to predict the results in EPl matches

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [10]:
df_epl = pd.read_csv(Path('Resources/epl_match_data.csv'),
                     parse_dates = ['DateTime'],
                     infer_datetime_format=True)
        

df_epl

,Unnamed: 0,Season,DateTime,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,Referee_l Mason,Referee_ A D'Urso,Referee_ A Wiley,Referee_ C Foy,Referee_ D Gallagher,Referee_ H Webb,Referee_ M Atkinson,Referee_ N Barry,Referee_ S Dunn,Referee_ U Rennie
0,0,2001,2000-08-19 00:00:00+00:00,4,0,2,2.0,0.0,2,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2001,2000-08-19 00:00:00+00:00,4,2,2,1.0,0.0,2,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2001,2000-08-19 00:00:00+00:00,1,3,0,1.0,1.0,1,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2001,2000-08-19 00:00:00+00:00,2,2,1,1.0,2.0,0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2001,2000-08-19 00:00:00+00:00,2,0,2,2.0,0.0,2,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,8284,2022,2022-04-09 17:30:00+00:00,0,4,0,0.0,1.0,0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8285,8285,2022,2022-04-10 14:00:00+00:00,2,0,2,0.0,0.0,1,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8286,8286,2022,2022-04-10 14:00:00+00:00,2,1,2,2.0,0.0,2,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8287,8287,2022,2022-04-10 14:00:00+00:00,2,0,2,1.0,0.0,2,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_epl = df_epl.drop(columns = ['Unnamed: 0'])
df_epl['DateTime'] = df_epl['DateTime'].apply(lambda x: x.toordinal())
df_epl.dtypes.value_counts()

float64    260
int64        6
dtype: int64

In [13]:
y = df_epl[['FTR']]
X= df_epl.drop(columns = ['FTR'])
display(y)
display(X)

,FTR
0,2
1,2
2,0
3,1
4,2
...,...
8284,0
8285,2
8286,2
8287,2


,Season,DateTime,FTHG,FTAG,HTHG,HTAG,HTR,HS,AS,HST,...,Referee_l Mason,Referee_ A D'Urso,Referee_ A Wiley,Referee_ C Foy,Referee_ D Gallagher,Referee_ H Webb,Referee_ M Atkinson,Referee_ N Barry,Referee_ S Dunn,Referee_ U Rennie
0,2001,730351,4,0,2.0,0.0,2,17.0,8.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2001,730351,4,2,1.0,0.0,2,17.0,12.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2001,730351,1,3,1.0,1.0,1,6.0,16.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2001,730351,2,2,1.0,2.0,0,6.0,13.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2001,730351,2,0,2.0,0.0,2,17.0,12.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,2022,738254,0,4,0.0,1.0,0,9.0,11.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8285,2022,738255,2,0,0.0,0.0,1,15.0,5.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8286,2022,738255,2,1,2.0,0.0,2,12.0,11.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8287,2022,738255,2,0,1.0,0.0,2,17.0,18.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(8289, 266)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
display(X_train.shape)

(6216, 265)

(2073, 265)

In [18]:
# Create the StandardScaler instance
X_scaler = MinMaxScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the the number of inputs to the model
number_inputs = 265

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 256

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 188


In [25]:
# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1))


In [26]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               68096     
                                                                 
 dense_3 (Dense)             (None, 188)               48316     
                                                                 
 dense_4 (Dense)             (None, 1)                 189       
                                                                 
Total params: 116,601
Trainable params: 116,601
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [28]:
# Fit the model
nn.fit(X_train_scaled,y_train, 
                    epochs=50,
                    batch_size=100,
                    shuffle=True)

Epoch 1/500
63/63 [==============================] - 2s 8ms/step - loss: 0.5287 - accuracy: 0.3069
Epoch 2/500
63/63 [==============================] - 0s 6ms/step - loss: 0.2541 - accuracy: 0.4077
Epoch 3/500
63/63 [==============================] - 0s 7ms/step - loss: 0.1531 - accuracy: 0.4583
Epoch 4/500
63/63 [==============================] - 0s 6ms/step - loss: 0.1083 - accuracy: 0.4949
Epoch 5/500
63/63 [==============================] - 0s 8ms/step - loss: 0.0829 - accuracy: 0.5188
Epoch 6/500
63/63 [==============================] - 1s 10ms/step - loss: 0.0614 - accuracy: 0.5310
Epoch 7/500
63/63 [==============================] - 0s 6ms/step - loss: 0.0471 - accuracy: 0.5375
Epoch 8/500
63/63 [==============================] - 1s 12ms/step - loss: 0.0390 - accuracy: 0.5396
Epoch 9/500
63/63 [==============================] - 0s 8ms/step - loss: 0.0267 - accuracy: 0.5415
Epoch 10/500
63/63 [==============================] - 0s 7ms/step - loss: 0.0191 - accuracy: 0.5418
Epoch 1